Problem set 2 (CS 490BH/890AG): Hidden Markov Models
=====================

- This project focuses on sequence labeling with Hidden Markov models.
- The target domain is Twitter part-of-speech tagging
- Submit everything in a zip folder on UR Courses, but don't use the rar extension, otherwise you will get zero grades

In [1]:
import numpy as np
from collections import defaultdict
from collections import defaultdict, Counter

%pylab --no-import-all inline
import gtnlplib.preproc
import gtnlplib.viterbi
import gtnlplib.most_common
import gtnlplib.naivebayes
import gtnlplib.clf_base
import gtnlplib.scorer
import gtnlplib.constants
import gtnlplib.tagger_base
import matplotlib.pyplot as plt
# this enables you to create inline plots in the notebook 
%pylab inline

Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


In [2]:
reload(gtnlplib.viterbi)
reload(gtnlplib.naivebayes)

<module 'gtnlplib.naivebayes' from 'gtnlplib\naivebayes.pyc'>

# 1. Data Processing (1 point) # 

The test data will be released around 48 hours before the deadline.
The part-of-speech tags are defined in the [ACL2011 paper](http://www.ark.cs.cmu.edu/TweetNLP/gimpel+etal.acl11.pdf) 
and the [NAACL 2013 paper](http://www.ark.cs.cmu.edu/TweetNLP/owoputi+etal.naacl13.pdf), 
which also describe the data and gives some state-of-art results.

In [3]:
## Define the file names
trainfile = gtnlplib.constants.TRAIN_FILE
devfile = gtnlplib.constants.DEV_FILE
testfile = gtnlplib.constants.TEST_FILE # You do not have this for now
offset = gtnlplib.constants.OFFSET

Here is a demo code for using function "conllSeqGenerator()"

In [4]:
## Demo
alltags = set()
for i,(words, tags) in enumerate(gtnlplib.preproc.conllSeqGenerator(trainfile)):    
    for tag in tags:
        alltags.add(tag)
print alltags


set(['!', '#', '$', '&', ',', 'A', '@', 'E', 'D', 'G', 'M', 'L', 'O', 'N', 'P', 'S', 'R', 'U', 'T', 'V', 'Y', 'X', 'Z', '^', '~'])


In [5]:
for (words, tags) in gtnlplib.preproc.conllSeqGenerator(trainfile):
    
    print words

['I', 'predict', 'I', "won't", 'win', 'a', 'single', 'game', 'I', 'bet', 'on', '.', 'Got', 'Cliff', 'Lee', 'today', ',', 'so', 'if', 'he', 'loses', 'its', 'on', 'me', 'RT', '@e_one', ':', 'Texas', '(', 'cont', ')', 'http://tl.gd/6meogh']
['RT', '@DjBlack_Pearl', ':', 'wat', 'muhfuckaz', 'wearin', '4', 'the', 'lingerie', 'party', '?????']
['Wednesday', '27th', 'october', '2010', '.', '\xe3\x80\x8bhave', 'a', 'nice', 'day', ':)']
['RT', '@ddlovato', ':', '@joejonas', 'oh', ',', 'hey', 'THANKS', 'jerk', '!']
['@thecamion', 'I', 'like', 'monkeys', ',', 'but', 'I', 'still', 'hate', 'COSTCO', 'parking', 'lots', '..']
['@DDaimaru', 'I', 'may', 'have', 'to', 'get', 'minecraft', 'after', 'watching', 'videos', 'of', 'it']
['RT', '@eye_ee_duh_Esq', ':', 'LMBO', '!', 'This', 'man', 'filed', 'an', 'EMERGENCY', 'Motion', 'for', 'Continuance', 'on', 'account', 'of', 'the', 'Rangers', 'game', 'tonight', '!', '\xc2\xab', 'Wow', 'lmao']
['RT', '@musicdenver', ':', 'Lady', 'Gaga', '-', 'Bad', 'Romance', 

['"', 'RUN', 'DAT', '"', 'by', '@IamPicclo', 'ft', '.', '@iitsAD', 'DOWNLOAD', 'NOW', '!!!!!', 'http://hulkshare.com/sr9ffmeald2q']
['I', 'can', 'hold', '75', 'cents', 'in', 'my', 'belly', 'button', ',', "that's", 'a', 'skill', 'I', 'can', 'take', 'straight', 'to', 'the', 'bank', '.']
['@alaxico', 'lmfao', 'that', 'different']
['RT', '@aurinasafitri', ':', 'RT', '@kaptenyoga', 'RT', '@merinamerinot', ':', 'RT', '@riskiielek', ':', 'RT', '@INDRAgnwn', ':', '*', 'm', 'RT', '@febianaisangga', ':', 'OPEN', 'FOLLOW', 'RTRTRT', '!!', '#JFB', '~', 'f', '...']
['Good', 'posture', 'can', 'help', 'avoid', 'back', 'problems', '-', 'so', 'sit', 'up', 'straight', '.', 'You', 'might', 'also', 'try', 'restorative', 'exercises', '.', 'http://ow.ly/306gD']
['I', 'bet', '!', 'Haha', '#teamceltics', 'yesterday', '!', 'Clowns', '!', 'RT', '@BigJon_99', ':', '#teamheat', 'at', 'Philly', '2nite', ',', "let's", 'go', '!']
['James', 'Bond\xe2\x80\x99s', 'Famous', 'Aston', 'DB5', 'to', 'be', 'Joined', 'by', '\

['These', 'gaps', 'really', 'motivate', 'me', 'to', 'make', 'things', 'happen', '...']
['MATCHES', 'YA', 'FACE', 'SO', 'YOU', 'COOL', '.', 'RT', '@iMsOdolly', ':', '@BaGzDaBoSS', 'its', 'growing', 'but', 'its', 'at', 'this', 'ugly', 'stage', ':(']
['RT', '@CashOutCassie', ':', 'd-town', '!']
['Wow', 'deep', 'lyrics', '"', 'when', 'you', 'smile', 'I', 'smile', '"']
['@crissaaa', 'come', 'bring', 'me', 'lunch', '!']
['I', "ain't", 'cheat', 'I', 'played', 'the', 'hand', 'I', 'dealt', '!']
['@chyeahJENNA', 'nerp', '!', 'Wish', 'I', 'could']
['@FelluzADOREmeeh', 'miss', 'yhuuuu', '!']
['@jethrova', 'perhaps']
['My', 'book', 'trailer', '-', 'WORST', 'SINGING', 'EVER', '!', 'YAY', '!:', 'http://t.co/cNxVMr3']
['I', 'hate', 'how', 'I', 'am', 'sometimes', ',', 'it', 'really', 'sucks', 'and', 'I', 'over', 'think', '.']
['Heading', 'to', 'Rock', 'n', 'Rave', 'but', 'can', 'the', 'bus', 'driver', 'find', 'it', '?', '@Rock', '-n-Rave', ',', 'check', 'it', 'out', "'"]
['Alaska', "judge's", 'order', 

**Deliverable 1a** (1 point): Use the Counter class to identify the most common three words for each tag, in the training set. The most_common() function of the Counter class will help you here. 

In [6]:
counters = gtnlplib.most_common.get_tags(trainfile)
for tag,tag_ctr in counters.iteritems():
    print tag,tag_ctr.most_common(3)

! [('lol', 54), ('Lol', 22), ('oh', 10)]
# [('#nowplaying', 3), ('#np', 2), ('#tcot', 2)]
$ [('one', 32), ('4', 6), ('2010', 6)]
& [('and', 117), ('&', 39), ('but', 31)]
, [('.', 427), ('!', 244), (',', 225)]
A [('good', 24), ('new', 22), ('more', 13)]
@ [('@Fresh32Prince89', 6), ('@lil_jeezy_85', 2), ('@ResourcefulMom', 2)]
E [(':)', 28), ('<3', 10), (';)', 8)]
D [('the', 236), ('a', 165), ('my', 89)]
G [('smh', 9), ('|', 7), ('-', 7)]
M [("momma's", 1), ('#LebronShould', 1), ("Ricochet's", 1)]
L [("I'm", 42), ('its', 24), ('im', 15)]
O [('I', 258), ('you', 135), ('it', 87)]
N [('day', 19), ('time', 18), ('people', 17)]
P [('to', 231), ('of', 112), ('for', 101)]
S [('mans', 1), ("judge's", 1), ('year\xe2\x80\x99s', 1)]
R [('just', 56), ('not', 27), ('now', 26)]
U [('http', 4), ('http://tumblr.com/xeinfgrpl', 1), (':/', 1)]
T [('out', 29), ('up', 26), ('on', 8)]
V [('is', 105), ('are', 52), ('have', 48)]
Y [("there's", 2)]
X [('all', 6), ('There', 4), ('there', 2)]
Z [("Obamacare's", 1

In [7]:
for tag,tag_ctr in counters.iteritems():
    for word in tag_ctr.keys():
        print(word, tag_ctr[word])

('Lmbo', 1)
('ugh', 3)
('yea', 6)
('YUP', 2)
('Dingdong', 1)
('derp', 1)
('hehehe', 1)
('Ayyye', 1)
('thanks', 1)
('hell', 1)
('yes', 4)
('Goodmorning', 1)
('hmmm', 1)
('otay', 1)
('foo', 2)
('ROAR', 1)
('Uhm', 1)
('HHHHHHHHHHHHHHHHHHHHHHi~\xe2\x98\x86', 1)
('lls', 1)
('l0l', 1)
('lmaoo', 2)
('wooo', 1)
('ohhh', 1)
('bye', 2)
('cahh', 1)
('AHHHHH', 1)
('goodmorning', 1)
('hahah', 1)
('noo', 1)
('Ok', 1)
('yo', 1)
('Oh', 6)
('ya', 4)
('#fuuuuuuucccit', 1)
('uhhh', 1)
('Hollaaaaa', 1)
('Right', 1)
('oke', 1)
('whatever', 1)
('GM', 1)
('uhmmmmm', 1)
('hjahah', 1)
('AHA', 1)
('OK', 1)
('OH', 2)
('guh', 1)
('OHMYGOODLORD', 1)
('Thanks', 1)
('lmfaoooooo', 1)
('night', 1)
('YES', 1)
('Unffffffff', 1)
('Iight', 1)
('Hiiiiya', 1)
('right', 3)
('4sho', 1)
('eh', 2)
('Hmmmmmm', 1)
('Wow', 6)
('yeah', 6)
('ewww', 2)
('Damn', 3)
('ahahha', 1)
('Umm', 1)
('omg', 3)
('HA', 2)
('SORRY', 1)
('nomnomnom', 1)
('Omg', 1)
('LMAO', 5)
('wow', 2)
('Haha', 6)
('Awww', 2)
('Wtf', 1)
('sup', 1)
('Awe', 1)
('Ha'

('LIL', 1)
('satisfied', 1)
('Same', 1)
('pansit-esque', 1)
('AMBER', 1)
('personal', 1)
('LONG', 1)
('late', 5)
('willing', 1)
('fade', 1)
('orange', 2)
('hidden', 1)
('earlyy', 1)
('good', 24)
('auto', 2)
('killer', 1)
('Bad', 2)
('Social', 1)
('front', 1)
('Senior', 2)
('contagious', 1)
('pregnant', 2)
('Sad', 3)
('Weekly', 1)
('financial', 1)
('hard', 3)
('related', 1)
('Next', 4)
('Figurative', 1)
('unlock', 1)
('year', 1)
('worried', 1)
('sexual', 1)
('special', 1)
('More', 4)
('secretly', 1)
('unlawful', 1)
('Downtown', 1)
('hilarious', 4)
('Authentic', 1)
('Cool', 1)
('Conscious', 1)
('fave', 1)
('free', 4)
('small', 1)
('Stupid', 1)
('Serious', 1)
('juiciest', 1)
('scary', 2)
('blahhh', 1)
('massive', 1)
('fuckin', 1)
('first', 4)
('major', 1)
('powerful', 1)
('spiritual', 1)
('one', 1)
('SILLY', 2)
('impossible', 1)
('open', 2)
('Excited', 1)
('little', 6)
('girly', 1)
('structured', 1)
('least', 1)
('white', 3)
('urban', 1)
('Dangerous', 1)
('ridiculous', 1)
('western', 1)
(

('E', 3)
('\xe2\x80\xba', 3)
('#', 1)
('wondering\xe2\x80\xa6is', 1)
('Y', 1)
('lar', 1)
('\xe2\x80\x94', 1)
('i', 1)
('sp', 1)
('m', 1)
('#goingtobedangry', 1)
('<<-----', 1)
("momma's", 1)
('#LebronShould', 1)
("Ricochet's", 1)
('Ima', 1)
("You're", 3)
('whats', 3)
("We'd", 1)
("He's", 2)
('Imma', 1)
("It's", 11)
("you've", 2)
('iv', 1)
("we'd", 1)
('wessss', 1)
('im', 15)
("We're", 1)
("Yo're", 1)
('id', 1)
('I\xe2\x80\x99ve', 1)
("I'mmm", 1)
('Id', 1)
('its', 24)
('ima', 1)
("you're", 10)
("how's", 1)
("he's", 6)
('imma', 1)
("it's", 12)
("What's", 2)
("Who's", 3)
('itss', 1)
('it\xe2\x80\x99s', 1)
("Where's", 2)
('#howcome', 2)
('lets', 1)
("it'd", 1)
('lemme', 3)
('lls', 1)
('they\xe2\x80\x99re', 1)
("Here's", 1)
('Wats', 1)
("i'z", 1)
("They're", 2)
("We'll", 1)
("i've", 1)
("IT'S", 1)
("what's", 5)
("who's", 1)
('Ur', 1)
("let's", 1)
("I'd", 1)
('Idk', 1)
('Whats', 1)
("I'm", 42)
('It\xe2\x80\x99s', 1)
("That's", 7)
("I'll", 3)
('#ilike', 1)
("we'll", 3)
("She'll", 1)
('your', 

('ON', 2)
('Because', 3)
('like', 36)
('OF', 3)
('Over', 2)
('CAUSe', 1)
('Before', 1)
('because', 4)
('From', 3)
('For', 1)
('w/', 3)
('Since', 1)
('Per', 1)
('are', 1)
('unless', 1)
('out', 12)
('for', 101)
('That', 1)
('since', 2)
('while', 3)
('below', 1)
('behind', 3)
('between', 1)
('cause', 2)
('Than', 2)
('FOR', 1)
('bc', 2)
('Against', 1)
('by', 19)
('on', 88)
('about', 23)
('befor', 1)
('of', 112)
('coz', 1)
('UP', 1)
('against', 2)
('Through', 1)
('w', 1)
('whole', 1)
('wif', 1)
('via', 9)
('into', 8)
('Cuz', 1)
('down', 2)
('wit', 4)
('By', 1)
('from', 38)
('next', 1)
('2', 12)
('fer', 1)
('thru', 1)
('until', 7)
('OUT', 1)
('that', 34)
('So', 7)
('FROM', 1)
('Beneath', 1)
('with', 51)
('(@', 4)
('ITIL', 1)
('LIKE', 1)
('inside', 1)
('up', 4)
('n', 2)
('near', 2)
('WIT', 2)
('SO', 1)
('toward', 1)
('outta', 1)
('On', 3)
('it', 1)
('To', 5)
('as', 18)
('vs', 1)
('at', 59)
('in', 97)
('Bout', 1)
('if', 29)
('Around', 1)
('til', 2)
('TO', 4)
('b/c', 1)
('With', 10)
('About', 1

('Hearing', 1)
('swear', 3)
('Shake', 1)
('cravin', 1)
('mean', 1)
('wanna', 7)
('enter', 2)
('plan', 1)
('AiNT', 1)
('seeing', 2)
('researching', 1)
('Cut', 1)
('followed', 1)
('wasting', 1)
('pays', 1)
('Make', 2)
('matter', 1)
('blessed', 1)
('socks', 1)
('guard', 1)
('noticedd', 1)
('raises', 1)
('insult', 1)
('OPEN', 1)
('smells', 1)
('grabbed', 1)
('behaving', 1)
('holla', 1)
('spending', 1)
('sneezing', 1)
('Hang', 1)
('styling', 1)
('noticed', 1)
('start', 9)
('pouts', 1)
('bored', 1)
('was', 48)
('MAKING', 1)
('#dontwait', 1)
('invite', 1)
('attempted', 1)
('becoming', 1)
('removes', 1)
('strikes', 1)
('suffers', 1)
('hear', 8)
('filed', 1)
('Following', 1)
('squeeze', 1)
('eat', 5)
('posted', 5)
('directed', 1)
('pull', 1)
('made', 5)
('procastinate', 1)
("wasn't", 3)
('wish', 3)
('\xe2\x99\xa5', 1)
('suuuuuuucks', 1)
('tellin', 1)
('taught', 1)
('invest', 1)
('called', 2)
('trim', 1)
('gone', 1)
('showin', 1)
('recognize', 1)
('am', 11)
('#fail', 1)
('Helppp', 1)
('Attack', 

In [8]:
for tag,tag_ctr in counters.iteritems():
   
    for word in tag_ctr:
        print([tag_ctr[word]] )
  

[1]
[3]
[6]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[4]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[6]
[4]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[2]
[1]
[6]
[6]
[2]
[3]
[1]
[1]
[3]
[2]
[1]
[1]
[1]
[5]
[2]
[6]
[2]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[6]
[1]
[4]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[7]
[10]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[2]
[3]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[10]
[54]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[10]
[1]
[1]
[2]
[1]
[1]
[1]
[4]
[5]
[1]
[1]
[1]
[4]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[8]
[1]
[22]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[1]
[1]
[2]
[3]
[1]
[1]
[4]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[3]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]

[1]
[2]
[1]
[1]
[2]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[1]
[1]
[2]
[1]
[1]
[4]
[1]
[3]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[3]
[2]
[1]
[1]
[2]
[1]
[1]
[2]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[3]
[2]
[1]
[3]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[13]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[4]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[2]
[2]
[1]
[2]
[1]
[3]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[2]
[2]
[9]
[1]
[1]
[2]
[2]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[6]
[3]
[2]
[1]
[1]
[6]
[1]
[1]
[1]
[1]
[1]
[1]
[7]
[1]
[5]
[1]
[1]
[1]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[2]
[1]
[19]
[1]
[1]
[1]
[1]
[1]
[1

[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[5]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[3]
[1]
[1]
[4]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[2]
[2]
[1]
[4]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[5]
[1]
[2]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[5]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[1]
[1]
[1]
[1]
[1]
[1]
[3]
[1]


# 2. Baseline models (5 points) # 

Now you will implement part-of-speech tagging via classification.

Tagging quality is evaluated using evalTagger, which takes three arguments:
- a tagger, which is a **function** taking a list of words and a tagset as arguments
- an output filename
- a test file

You will want to use lambda expressions to create the first argument for this function, as shown below.
Here's how it works. I provide a tagger that labels everything as a noun.

In [9]:
# here is a tagger that just tags everything as a noun
noun_tagger = lambda words, alltags : ['N' for word in words]
confusion = gtnlplib.tagger_base.evalTagger(noun_tagger,'nouns')
print gtnlplib.scorer.accuracy(confusion)

0.136844287788


In [10]:
confusion

defaultdict(int,
            {('!', 'N'): 99,
             ('#', 'N'): 52,
             ('$', 'N'): 86,
             ('&', 'N'): 91,
             (',', 'N'): 500,
             ('@', 'N'): 243,
             ('A', 'N'): 239,
             ('D', 'N'): 312,
             ('E', 'N'): 52,
             ('G', 'N'): 65,
             ('L', 'N'): 65,
             ('N', 'N'): 660,
             ('O', 'N'): 333,
             ('P', 'N'): 440,
             ('R', 'N'): 209,
             ('S', 'N'): 5,
             ('T', 'N'): 36,
             ('U', 'N'): 91,
             ('V', 'N'): 751,
             ('X', 'N'): 4,
             ('Z', 'N'): 9,
             ('^', 'N'): 311,
             ('~', 'N'): 170})

In [11]:
print(noun_tagger)

<function <lambda> at 0x000000000C63FC18>


** Deliverable 2a ** (1 point)

Now do the same thing, but building your tagger *as a classifier.* To do this:

- use makeClassifierTagger, which takes as an argument a dict of weights
- set the weights yourself, by filling in gtnlplib.most_common.get_noun_weights()

In [12]:
cTagger = gtnlplib.tagger_base.makeClassifierTagger(gtnlplib.most_common.get_noun_weights())

In [13]:
gtnlplib.most_common.get_noun_weights()

defaultdict(int, {('N', '**OFFSET**'): 1})

In [14]:
gtnlplib.most_common.get_noun_weights().has_key(('N', '**OFFSET**'))

True

In [15]:
cTagger


<function gtnlplib.tagger_base.<lambda>>

In [16]:
confusion = gtnlplib.tagger_base.evalTagger(cTagger,'nouns')
print gtnlplib.scorer.accuracy(confusion)

0.136844287788


**Deliverable 2b** (2 points)

Now build a classifier tagger that tags each word with its most common tag in the training set.

- You should again implement your classifier by defining a set of weights
- Prediction should use your predict() function from pset 1. (you are allowed to edit this function if you don't think you got it right in pset 1.)
- For unseen words, the classifier should choose the tag with the most **unique** word types.

In [17]:
weights = gtnlplib.most_common.get_most_common_weights(gtnlplib.constants.TRAIN_FILE)
confusion = gtnlplib.tagger_base.evalTagger(gtnlplib.tagger_base.makeClassifierTagger(weights),'mcc')
print gtnlplib.scorer.accuracy(confusion)

0.631349782293


In [18]:
seen = False
for tag in tags:
    seen = weights.has_key((tag,word))
    print seen

True
True
True
True
True
True
True
True
True


In [19]:
weights

defaultdict(int,
            {('P', 'asses'): 0,
             ('G', 'says'): 0,
             ('Y', 'read'): 0,
             ('V', 'flex'): 1,
             ('R', 'hit'): 0,
             ('L', 'like'): 0,
             ('X', 'give'): 0,
             ('#', 'coming'): 0,
             ('D', 'AND'): 0,
             ('D', ':'): 0,
             ('#', 'ice'): 0,
             ('^', 'Burdick'): 1,
             ('V', 'Tastes'): 1,
             ('S', 'commercial'): 0,
             ('N', 'Ave'): 0,
             ('L', 'world'): 0,
             ('@', 'fat'): 0,
             ('N', 'more'): 0,
             ('^', 'heechul'): 1,
             ('V', 'blood'): 0,
             ('L', 'asking'): 0,
             ('Y', 'every'): 0,
             ('!', 'jk'): 2,
             ('^', 'bitches'): 0,
             ('O', 'talking'): 0,
             ('D', 'Risk'): 0,
             ('G', 'iDC'): 1,
             ('^', 'grants'): 0,
             ('R', 'need'): 0,
             ('A', 'Happy'): 1,
             ('!', 'lmao'): 5,
  

In [20]:
confusion

defaultdict(int,
            {('!', '!'): 53,
             ('!', ','): 2,
             ('!', 'D'): 2,
             ('!', 'N'): 4,
             ('!', 'O'): 1,
             ('!', 'R'): 2,
             ('!', 'Y'): 35,
             ('#', '#'): 6,
             ('#', 'Y'): 45,
             ('#', '^'): 1,
             ('$', '$'): 30,
             ('$', 'A'): 1,
             ('$', 'O'): 1,
             ('$', 'P'): 5,
             ('$', 'Y'): 48,
             ('$', '^'): 1,
             ('&', '&'): 87,
             ('&', 'D'): 2,
             ('&', 'Y'): 2,
             (',', ','): 462,
             (',', 'Y'): 7,
             (',', '~'): 31,
             ('@', '@'): 6,
             ('@', 'Y'): 237,
             ('A', '!'): 2,
             ('A', '$'): 1,
             ('A', 'A'): 130,
             ('A', 'N'): 6,
             ('A', 'R'): 6,
             ('A', 'T'): 1,
             ('A', 'V'): 4,
             ('A', 'Y'): 89,
             ('D', '!'): 2,
             ('D', 'D'): 292,
             ('

**Deliverable 2c** (1 point)

Now use your function ```learnNBWeights``` from pset 2 to set the weights in the classifier-tagger.

You will need feature-class counts (where there is one feature per tag: the word), and class counts.

In [21]:
# build a list of all words
allwords = set()
for counts in counters.values():
    print(allwords.update(set(counts.keys())))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [22]:
class_counts = gtnlplib.most_common.get_class_counts(counters)

In [23]:
w1 = gtnlplib.naivebayes.learnNBWeights(counters,class_counts,allwords)
print w1[('N','breakfast')], w1[('V','breakfast')], w1[('A','smart')], w1[('D','the')], w1[('!',gtnlplib.constants.OFFSET)]
print w1[99]

-7.74704715082322 -10.226367835637873 -6.426796623869991 -1.7886486069907792 -3.5837241719098833
0


**Deliverable 2d** (1 point): run the code below to evaluate your naive bayes tagger on the development set

In [24]:
dev_acc = dict()
for alpha in [1e-4,1e-3,1e-2,1e-1,1e0,1e1,1e3]:
    nb_weights = gtnlplib.naivebayes.learnNBWeights(counters,class_counts,allwords,alpha)
    confusion = gtnlplib.tagger_base.evalTagger(gtnlplib.tagger_base.makeClassifierTagger(nb_weights),'nb')
    dev_acc[alpha] = gtnlplib.scorer.accuracy(confusion)
    print alpha,dev_acc[alpha]

0.0001 0.614762595895
0.001 0.614762595895
0.01 0.622226829774
0.1 0.629691063653
1.0 0.630520422973
10.0 0.630935102633
1000.0 0.630935102633


# 3 Viterbi Algorithm (10 points) #

In this section you will implement the Viterbi algorithm. As a reminder, here it is:

\begin{align*}
\vec{w}'\vec{f}(\vec{x},\vec{y}) = & \sum_i \vec{w}^{\top} \vec{f}(\vec{x},y_i,y_{i-1},i) \\
v(y,0) = & \vec{w}^{\top}\vec{f}(\vec{x},y,\diamond,0)\\
b(y,0) = & \diamond \\
v(y,i) = & \max_{y'} \vec{w}^{\top}\vec{f}(\vec{x},y,y',i) + v(y',i-1)\\
b(y,i-1) = & \text{arg}\max_{y'} \vec{w}^{\top}\vec{f}(\vec{x},y,y',i) + v(y',i-1)\\
\end{align*}

To get warmed up, let's work out an example by hand. These are only two tags, 
N and V. Here are the parameters:

| | Value |
| ------------- |:-------------:|
| $\log P_E(\cdot|N)$ | they: -1, can: -3, fish: -3 |
| $\log P_E(\cdot|V)$ | they: -10, can: -2, fish: -3 |
| $\log P_T(\cdot|N)$ | N: -5, V: -2, END: -2 |
| $\log P_T(\cdot|V)$ | N: -1, V: -4, END: -3 |
| $\log P_T(\cdot|\text{START})$ | N :-1, V :-1 |

where $P_E(\cdot|\cdot)$ is the emission probability and $P_T(\cdot|\cdot)$ is the translation probability.
 
In class we discuss the sentence *They can fish*. Now work out a more complicated example: "*They can can fish*".
 
** Deliverable 3a ** (2 points) Show the trellis-like table, and give the score for the best best scoring path(s). After you work out the trellis by hand, you should be able to fill the following table.


** Sanity check ** There are two paths that each score -18.

*(Fill your answer in the following table)*

|POS tag| START  | they | can | can | fish | END |
|-------|:-------|:-----|:----|:----|:-----|:---:|
| N     |    0   |    |  |  |   |  |
| V     |    0   |   |   |  |   |  |

## Implementing Viterbi ##

Here are some predefined weights, corresponding to problem 3a.

In [25]:
start_tag = gtnlplib.constants.START_TAG
trans = gtnlplib.constants.TRANS
end_tag = gtnlplib.constants.END_TAG
emit = gtnlplib.constants.EMIT

defined_weights = {('N','they',emit):-1,('N','can',emit):-3,('N','fish',emit):-3,\
                        ('V','they',emit):-10,('V','can',emit):-2,('V','fish',emit):-3,\
                        ('N','N',trans):-5,('V','N',trans):-2,(end_tag,'N',trans):-3,\
                        ('N','V',trans):-1,('V','V',trans):-4,(end_tag,'V',trans):-3,\
                        ('N',start_tag,trans):-1,('V',start_tag,trans):-1}

In [26]:
trainfile = gtnlplib.constants.TRAIN_FILE

`gtnlplib.viterbi.hmm_feats` computes the HMM features for the function $\vec{f}(\vec{x},y,y',i)$. 
- You will call it in your viterbi tagger. 
- Note that it returns both an emission and transition feature, except for the last word, where it returns only a transition feature. 
- Also note that transition and emission features are specially marked

**Deliverable 3b** (5 points)

Implement `viterbiTagger` in `gtnlplib/viterbi.py`

- **Input 1**: a list of words
- **Input 2**: a feature function, like hmm_feats
- **Input 3**: a dict of weights
- **Input 4**: a list of all possible tags
- **Output 1**: the best-scoring sequence
- **Output 2**: the score of the best-scoring sequence

Hint: you can represent the trellis and the back pointers as lists of dicts. You will want to do some special handling for the first and last words; otherwise, just iterate 

Run you viterbi tagger on the example in 3a, using the code below.

In [27]:
def hmm_feats(words,curr_tag,prev_tag,i):
    """Feature function for HMM that returns emit and transition features"""
    if i < len(words):
        return [(curr_tag,words[i],emit),(curr_tag,prev_tag,trans)]
    else:
        return [(curr_tag,prev_tag,trans)]

In [28]:
gtnlplib.viterbi.viterbiTagger(['they','can','can','fish'],hmm_feats,defined_weights,['N','V'])

(['N', 'V', 'N', 'V'], -18)

Run your Viterbi on the following example:

In [29]:
sent = 'they can can can can can can can fish'.split()

In [30]:
gtnlplib.viterbi.viterbiTagger(sent,hmm_feats,defined_weights,['N','V'])

(['N', 'V', 'N', 'V', 'N', 'V', 'N', 'V', 'N'], -37)

** Deliverable 3c**
(2 points)

Now estimate the weights of a hidden Markov model. 
- You have already estimated the emission weights $\log P(w | y)$, in your solution to problem 2. Use your solution with $\alpha=0.001$
- Estimate the transition probabilities from the training data, using the maximum-likelihood estimates (no smoothing)
- Don't forget transitions from the start state and to the end state

In [31]:
reload(gtnlplib.viterbi)
reload(gtnlplib.naivebayes)

<module 'gtnlplib.naivebayes' from 'gtnlplib\naivebayes.pyc'>

In [32]:
hmm_weights = gtnlplib.viterbi.get_HMM_weights(trainfile)

gtnlplib\viterbi.py:112: RuntimeWarning: divide by zero encountered in log
  trans_weights[(t_tag, tag)] = np.log(float(trans_counter[(t_tag, tag)])/count) #num of t_tag after tag divided by total words in that tag
gtnlplib\viterbi.py:114: RuntimeWarning: divide by zero encountered in log
  trans_weights[(END_TAG,tag)] = np.log(float(trans_counter[(END_TAG,tag)])/num_insts) #prob of each ending tag
gtnlplib\viterbi.py:113: RuntimeWarning: divide by zero encountered in log
  trans_weights[(tag,START_TAG) ] = np.log(float(trans_counter[(tag, START_TAG) ])/num_insts) #prob of each starting tag


In [33]:
print hmm_weights['V','go',emit], hmm_weights['~','go',emit], hmm_weights['^','diddy',emit]
print hmm_weights['V','V',trans], hmm_weights['~','V',trans]
print hmm_weights[end_tag,'V',trans], hmm_weights[end_tag,'~',trans]

-4.874509702161808 -9.288411907396338 -1000.0
-1.9179145415658865 -5.913052453704539
-3.506557897319982 -3.6888794541139363


**Sanity check**: here's the tag sequence and score for our example sentence

In [34]:
gtnlplib.viterbi.viterbiTagger([':))','we','can','can','fish',':-)'],hmm_feats,hmm_weights,alltags)

(['E', 'O', 'V', 'V', 'N', ','], -52.49549198179622)

In [41]:
hmm_weights2 = gtnlplib.viterbi.get_HMM_weights(devfile)

** Deliverable 3d** (1 point):
- Run your HMM tagger on the dev data, using the code line below.
- ** Sanity check**: I get 74.5% accuracy

In [37]:
for i,(words,_) in enumerate(gtnlplib.preproc.conllSeqGenerator(devfile)):
    print i, gtnlplib.viterbi.viterbiTagger(words,hmm_feats,hmm_weights,alltags)
    if i >= 2: break

0 (['V', 'O', 'V', 'P', 'N', 'N', ',', '$', 'P', 'D', 'N', '&', 'D', 'N', 'N', 'N', ','], -5081.850497423216)
1 (['~', '@', '~', 'O', 'V', 'P', '^', '^', ','], -4031.5714418923335)
2 (['~', '@', '~', '^', '^', '$', 'N', 'V', 'P', 'D', 'N', 'N', 'P', 'D', 'N', 'P', 'N', 'P', 'D', 'N', 'P', 'D', 'N', 'N', 'N', ','], -5162.464723963295)


In [38]:
confusion = gtnlplib.tagger_base.evalTagger(lambda words, alltags : gtnlplib.viterbi.viterbiTagger(words,hmm_feats,hmm_weights,alltags)[0],'hmm')
print gtnlplib.scorer.accuracy(confusion)

0.720091229525


In [45]:
for i,(words,_) in enumerate(gtnlplib.preproc.conllSeqGenerator(trainfile)):
    print i, gtnlplib.viterbi.viterbiTagger(words,hmm_feats,hmm_weights2,alltags)
    if i >= 2: break

0 (['O', 'V', 'O', 'V', 'V', 'D', 'N', 'N', 'O', 'V', 'P', ',', 'O', 'V', 'D', 'N', ',', 'R', 'P', 'O', 'V', 'T', 'P', 'O', '~', '@', '~', '^', ',', '!', ',', 'U'], -11144.636025520047)
1 (['~', '@', '~', 'O', 'V', 'V', 'P', 'D', 'N', 'N', ','], -5043.280655294356)
2 (['^', '$', 'N', 'N', ',', 'V', 'D', 'A', 'N', 'E'], -4046.996848735604)


In [46]:
confusion = gtnlplib.tagger_base.evalTagger(lambda words, alltags : gtnlplib.viterbi.viterbiTagger(words,hmm_feats,hmm_weights2,alltags)[0],'hmm')
print gtnlplib.scorer.accuracy(confusion)

0.916234708688


# 4. Making it better #

Find an example of expectation-maximization in use in a paper at ACL, NAACL, EMNLP, EACL, or TACL, within the last five years (2016-2021). List:

- The title, authors, and venue of the paper
- What is the "missing data" (latent variable) that they are imputing in the E-step?
- What are the parameters that they are trying to estimate?
- Do they take any steps to correct for local optima?

# Credit #

This assignment was originally developed by Jacob Eisenstein at Georgia Institute of Technology.